In [2]:
import warnings
warnings.filterwarnings('ignore', message='urllib3 v2 only supports OpenSSL')
import holidays 
import pandas as pd
import random
import requests


# 1. Identify the Germany Bank Holidays
Bank Holidays need to be avoided in our synthesize data

In [3]:
german_hes_holidays = holidays.country_holidays('DE', subdiv='HE', years=[2025])
print(german_hes_holidays)

{datetime.date(2025, 1, 1): 'Neujahr', datetime.date(2025, 4, 18): 'Karfreitag', datetime.date(2025, 4, 21): 'Ostermontag', datetime.date(2025, 5, 1): 'Erster Mai', datetime.date(2025, 5, 29): 'Christi Himmelfahrt', datetime.date(2025, 6, 9): 'Pfingstmontag', datetime.date(2025, 10, 3): 'Tag der Deutschen Einheit', datetime.date(2025, 12, 25): 'Erster Weihnachtstag', datetime.date(2025, 12, 26): 'Zweiter Weihnachtstag', datetime.date(2025, 6, 19): 'Fronleichnam'}


# 2. Create a Dataframe with all Days
Create List Mon-Fr, starting in Jan 24 through to Oct 25 with Weekdays

In [8]:
all_dates = pd.date_range(start='2024-01-01', end='2025-10-31', freq='B')
df = pd.DataFrame({
    'date': all_dates,
    'day_of_week': all_dates.day_name(),
    'month': all_dates.month,
})
print(df)



          date day_of_week  month
0   2024-01-01      Monday      1
1   2024-01-02     Tuesday      1
2   2024-01-03   Wednesday      1
3   2024-01-04    Thursday      1
4   2024-01-05      Friday      1
..         ...         ...    ...
475 2025-10-27      Monday     10
476 2025-10-28     Tuesday     10
477 2025-10-29   Wednesday     10
478 2025-10-30    Thursday     10
479 2025-10-31      Friday     10

[480 rows x 3 columns]


# 3. Label Days that are German Holiday
Label the german bank holidays in the dataframe

In [9]:
for index, row in df.iterrows():
    if row['date'] in german_hes_holidays:
        df.at[index, 'is_holiday'] = True
    else:
        df.at[index, 'is_holiday'] = False
print(df)

          date day_of_week  month is_holiday
0   2024-01-01      Monday      1       True
1   2024-01-02     Tuesday      1      False
2   2024-01-03   Wednesday      1      False
3   2024-01-04    Thursday      1      False
4   2024-01-05      Friday      1      False
..         ...         ...    ...        ...
475 2025-10-27      Monday     10      False
476 2025-10-28     Tuesday     10      False
477 2025-10-29   Wednesday     10      False
478 2025-10-30    Thursday     10      False
479 2025-10-31      Friday     10      False

[480 rows x 4 columns]


# 4. Remove German Holidays
Filter by is_holiday == False

In [10]:
df_filtered = df[df['is_holiday'] == False]
print(df_filtered)

          date day_of_week  month is_holiday
1   2024-01-02     Tuesday      1      False
2   2024-01-03   Wednesday      1      False
3   2024-01-04    Thursday      1      False
4   2024-01-05      Friday      1      False
5   2024-01-08      Monday      1      False
..         ...         ...    ...        ...
475 2025-10-27      Monday     10      False
476 2025-10-28     Tuesday     10      False
477 2025-10-29   Wednesday     10      False
478 2025-10-30    Thursday     10      False
479 2025-10-31      Friday     10      False

[462 rows x 4 columns]


# 5. Add Capacity 
Max capacity is 400 and will vary based on the weekday

In [13]:

for index, row in df_filtered.iterrows():
    if row['day_of_week'] == 'Monday': 
        df_filtered.at[index, 'capacity'] = random.randint(200, 300)
    elif row['day_of_week'] == 'Friday':
        df_filtered.at[index, 'capacity'] = random.randint(150, 250)
    elif row['day_of_week'] == 'Wednesday':
        df_filtered.at[index, 'capacity'] = random.randint(180, 280)
    else:
        df_filtered.at[index, 'capacity'] = random.randint(300, 400)
print(df_filtered)  

          date day_of_week  month is_holiday  capacity
1   2024-01-02     Tuesday      1      False     327.0
2   2024-01-03   Wednesday      1      False     186.0
3   2024-01-04    Thursday      1      False     386.0
4   2024-01-05      Friday      1      False     227.0
5   2024-01-08      Monday      1      False     284.0
..         ...         ...    ...        ...       ...
475 2025-10-27      Monday     10      False     249.0
476 2025-10-28     Tuesday     10      False     350.0
477 2025-10-29   Wednesday     10      False     275.0
478 2025-10-30    Thursday     10      False     351.0
479 2025-10-31      Friday     10      False     179.0

[462 rows x 5 columns]


# 5. Add Temperature
Using Open AI Weather to query historical data

In [14]:
url = "https://archive-api.open-meteo.com/v1/archive"
params =  {
    "latitude": 50.1330,
    "longitude": 8.6807,
    "start_date": "2024-01-01",
    "end_date": "2025-10-31",
    "timezone": "Europe/Berlin",
    "daily": "temperature_2m_max",
}   
r = requests.get(url, params=params)
print(r.url)
response_code = r.status_code
print(f"Response Code: {response_code}")
weather_data = r.json()

# Extract the actual weather data
if 'daily' in weather_data:
    daily_data = weather_data['daily']
    dates = daily_data['time']
    temperatures = daily_data['temperature_2m_max']
    
    # Create a dataframe with the weather data
    weather_df = pd.DataFrame({
        'date': pd.to_datetime(dates),
        'temperature_max': temperatures
    })
    print(weather_df.head(20))


https://archive-api.open-meteo.com/v1/archive?latitude=50.133&longitude=8.6807&start_date=2024-01-01&end_date=2025-10-31&timezone=Europe%2FBerlin&daily=temperature_2m_max
Response Code: 200
         date  temperature_max
0  2024-01-01              7.6
1  2024-01-02              9.4
2  2024-01-03             10.7
3  2024-01-04              8.4
4  2024-01-05              8.9
5  2024-01-06              6.9
6  2024-01-07              4.0
7  2024-01-08             -1.2
8  2024-01-09             -0.1
9  2024-01-10              0.2
10 2024-01-11              2.5
11 2024-01-12              0.8
12 2024-01-13              2.2
13 2024-01-14              0.2
14 2024-01-15              2.6
15 2024-01-16              1.6
16 2024-01-17              0.6
17 2024-01-18              1.3
18 2024-01-19              0.2
19 2024-01-20             -2.0


# 6. Merge the Temperature Data

In [15]:
result = pd.merge(df_filtered, weather_df, how="left", on="date")
print(result)

          date day_of_week  month is_holiday  capacity  temperature_max
0   2024-01-02     Tuesday      1      False     327.0              9.4
1   2024-01-03   Wednesday      1      False     186.0             10.7
2   2024-01-04    Thursday      1      False     386.0              8.4
3   2024-01-05      Friday      1      False     227.0              8.9
4   2024-01-08      Monday      1      False     284.0             -1.2
..         ...         ...    ...        ...       ...              ...
457 2025-10-27      Monday     10      False     249.0             11.7
458 2025-10-28     Tuesday     10      False     350.0             12.5
459 2025-10-29   Wednesday     10      False     275.0             14.8
460 2025-10-30    Thursday     10      False     351.0             13.1
461 2025-10-31      Friday     10      False     179.0             11.4

[462 rows x 6 columns]


# 6. Meal Count
Add PoS based on capacity, weekday and temperature using a % value

In [22]:
for index, row in result.iterrows():
    multiplier = random.uniform(0.5,0.75)
    result.at[index, 'meals'] = round(row['capacity']*multiplier)
print(result) 

          date day_of_week  month is_holiday  capacity  temperature_max  meals
0   2024-01-02     Tuesday      1      False     327.0              9.4  207.0
1   2024-01-03   Wednesday      1      False     186.0             10.7  112.0
2   2024-01-04    Thursday      1      False     386.0              8.4  252.0
3   2024-01-05      Friday      1      False     227.0              8.9  127.0
4   2024-01-08      Monday      1      False     284.0             -1.2  192.0
..         ...         ...    ...        ...       ...              ...    ...
457 2025-10-27      Monday     10      False     249.0             11.7  175.0
458 2025-10-28     Tuesday     10      False     350.0             12.5  196.0
459 2025-10-29   Wednesday     10      False     275.0             14.8  147.0
460 2025-10-30    Thursday     10      False     351.0             13.1  260.0
461 2025-10-31      Friday     10      False     179.0             11.4  121.0

[462 rows x 7 columns]


# 7. Save data as csv file

In [23]:
result.to_csv('test_data.csv')